#Instalaciones e importaciones

In [ ]:
!pip -q install datasets tiktoken openai==0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
import openai
import os
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

openai.api_key = "API KEY PERSONAL DE OPEN AI"

#Subiendo archivos de dataset a openai
Nota: se suben los archivos y se les asignan unos ID, es importante guardar esos ID

In [ ]:
training_file_name = 'https://raw.githubusercontent.com/Beta-sebas/dataset_1/main/amadeus_TRL_DATASET_GPT3.5TURBO_VALIDATION.jsonl'
validation_file_name = 'https://raw.githubusercontent.com/Beta-sebas/dataset_1/main/amadeus_TRL_DATASET_GPT3.5TURBO_VALIDATION.jsonl'

In [ ]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-o0LxbyV2fW7jy4pO4Eyny6bY
Validation file id: file-HtDITvXKmygBMNKTsjCTwOyh


#Se crea un Finetuning Job

In [ ]:
suffix_name = "AMADEUS-V2"


response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-qOfQ3xM0zHotzKPNJ9eG0vGi",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1716826784,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-NSATp5KWNUBkctvLTflUhjwL",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-HtDITvXKmygBMNKTsjCTwOyh",
  "training_file": "file-o0LxbyV2fW7jy4pO4Eyny6bY",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": "AMADEUS-V2",
  "seed": 1808061051,
  "estimated_finish": null,
  "integrations": []
}


In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-qOfQ3xM0zHotzKPNJ9eG0vGi",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1716826784,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-NSATp5KWNUBkctvLTflUhjwL",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-HtDITvXKmygBMNKTsjCTwOyh",
  "training_file": "file-o0LxbyV2fW7jy4pO4Eyny6bY",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": "AMADEUS-V2",
  "seed": 1808061051,
  "estimated_finish": null,
  "integrations": []
}


In [ ]:
response = openai.FineTuningJob.list_events(id=job_id, limit=100)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


Step 1159/1254: training loss=0.56
Step 1160/1254: training loss=0.34
Step 1161/1254: training loss=0.65
Step 1162/1254: training loss=0.58
Step 1163/1254: training loss=0.34
Step 1164/1254: training loss=0.61
Step 1165/1254: training loss=0.78
Step 1166/1254: training loss=0.48
Step 1167/1254: training loss=0.53
Step 1168/1254: training loss=0.60
Step 1169/1254: training loss=0.51
Step 1170/1254: training loss=0.37
Step 1171/1254: training loss=0.39
Step 1172/1254: training loss=0.46
Step 1173/1254: training loss=0.35
Step 1174/1254: training loss=0.34
Step 1175/1254: training loss=0.30
Step 1176/1254: training loss=0.33
Step 1177/1254: training loss=0.24
Step 1178/1254: training loss=0.77
Step 1179/1254: training loss=0.75
Step 1180/1254: training loss=0.16
Step 1181/1254: training loss=1.16
Step 1182/1254: training loss=0.15
Step 1183/1254: training loss=0.57
Step 1184/1254: training loss=0.34
Step 1185/1254: training loss=1.03
Step 1186/1254: training loss=0.53
Step 1187/1254: trai

#Probar el modelo ya entrenado

Pregunta:

In [ ]:
test_messages = []
system_job = "Eres un asistente de análisis de madurez de tecnolgías"
test_messages.append({"role": "system", "content": system_job})
user_message = "qué necesito para llegar a TRL7?"
test_messages.append({"role": "user", "content": user_message})

for message in test_messages:
  print(message)

{'role': 'system', 'content': 'Eres un asistente de recetas de comida'}
{'role': 'user', 'content': 'como puedo hacer que mi pan crezca?'}


Respuesta del modelo entrenado:

In [ ]:
fine_tuned_model_id = "id del modelo recién creado"
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0.5, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

 Para hacer que tu pan crezca, asegúrate de que la levadura esté fresca y activa. También es importante amasar bien la masa para desarrollar el gluten. Además, déjalo reposar en un lugar cálido y sin corrientes de aire para que fermente adecuadamente.


Respuesta del modelo original:

In [ ]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo', messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

Para hacer que tu pan crezca adecuadamente, aquí tienes algunos consejos:

1. Utiliza levadura fresca o activa: Asegúrate de que tu levadura esté fresca y activa. Si usas levadura seca activa, disuélvela en agua tibia con una pizca de azúcar y déjala reposar durante unos minutos hasta que se forme una espuma.

2. Amasa correctamente: Amasa la masa lo suficiente para desarrollar el gluten. Esto ayudará a retener el dióxido de carbono producido por la levadura y permitirá que el pan crezca.

3. Deja que la masa repose y fermente: Después de amasar, deja que la masa repose en un lugar cálido y sin corrientes de aire para que fermente. Esto permitirá que la levadura haga su trabajo y haga que la masa crezca.

4. Proporciona un ambiente cálido: Asegúrate de que la temperatura ambiente sea adecuada para que la levadura funcione correctamente. Si hace frío, puedes colocar la masa en un lugar cálido, como cerca de una fuente de calor o en el horno apagado con la luz encendida.

5. No agregues 

Capturo la primera respuesta del asistente para establecer el historial de interacciones en los mensajes:

In [ ]:
test_messages.append({"role": "assistant", "content": response["choices"][0]["message"]["content"]})
user_message = "y cuales son sus desventajas?"
test_messages.append({"role": "user", "content": user_message})
for message in test_messages:
  print(message)

{'role': 'system', 'content': 'Eres un asistente de Platzi, que es una plataforma de cursos de educación en línea. Tú ayudas a sus estudiantes a resolver dudas sobre la plataforma y sus cursos'}
{'role': 'user', 'content': 'cuales son las ventajas de aprender PHP?'}
{'role': 'assistant', 'content': ' Algunas ventajas de aprender PHP son: su popularidad y demanda en el mercado laboral, su facilidad de uso y aprendizaje, su amplia comunidad de desarrolladores, su compatibilidad con diferentes sistemas operativos y su integración con bases de datos.'}
{'role': 'user', 'content': 'y cuales son sus desventajas?'}


Solicito la segunda interacción del asistente en este hilo de conversación

In [ ]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

 Algunas desventajas de PHP son: su reputación de ser un lenguaje menos seguro en comparación con otros, su falta de soporte para programación orientada a objetos en versiones anteriores, y su rendimiento relativamente más lento en comparación con otros lenguajes. Sin embargo, muchas de estas desventajas han sido abordadas en versiones más recientes de PHP.
